In [1]:
import pandas as pd

In [2]:
#버퍼안에 속한 50m 격자 정보 불러옴
buffer = pd.read_csv('버퍼안 50m격자.csv')
buffer.head(1)

,cell_50_id,cell_250_i,x_katech,y_katech,gu_nm,hj_dong_nm,역
0,59733232,다바91bb15ba,347202,415343,중구,목동,중촌


In [25]:
#11.30 파일 데이터 크기가 수깁기가 넘어서거나 계속 정제하면 램부족등으로 재부팅 해줘야함
#특정달 15일 데이터
#data = pd.read_csv('DAEJEON_20190101_20190115.csv')#1월
#data = pd.read_csv('DAEJEON_20190201_20190215.csv')#2월
#data = pd.read_csv('DAEJEON_20190301_20190315.csv')#3월
#data = pd.read_csv('DAEJEON_20190401_20190415.csv')#4월
#data = pd.read_csv('DAEJEON_20190501_20190515.csv')#5월
#data = pd.read_csv('DAEJEON_20190601_20190615.csv')#6월
#data = pd.read_csv('DAEJEON_20190701_20190715.csv')#7월
#data = pd.read_csv('DAEJEON_20190801_20190815.csv')#8월
#data = pd.read_csv('DAEJEON_20190901_20190915.csv')#9월
#data = pd.read_csv('DAEJEON_20191001_20191015.csv')#10월
#data = pd.read_csv('DAEJEON_20191101_20191115.csv')#11월
data = pd.read_csv('DAEJEON_20191201_20191215.csv')#12월
data.head()

,id,x,y,timezn_cd,m00,m10,m15,m20,m25,m30,...,f35,f40,f45,f50,f55,f60,f65,f70,total,etl_ymd
0,56173942,332602,410243,0,0,0,0,1,0,1,...,1,1,0,1,0,0,0,0,16,20191204
1,56173942,332602,410243,0,0,0,0,1,1,1,...,1,1,1,1,0,0,0,0,13,20191213
2,56173942,332602,410243,0,0,0,0,2,1,1,...,1,1,1,1,1,0,0,0,17,20191215
3,56173942,332602,410243,0,0,0,1,1,1,1,...,0,0,1,0,0,0,0,0,10,20191202
4,56173942,332602,410243,0,0,0,1,1,1,1,...,0,1,1,1,0,0,0,0,16,20191206


In [26]:
#특정달 16~말일 데이터
#data2 = pd.read_csv('DAEJEON_20190116_20190131.csv')#1월
#data2 = pd.read_csv('DAEJEON_20190216_20190228.csv')#2월
#data2 = pd.read_csv('DAEJEON_20190316_20190331.csv')#3월
#data2 = pd.read_csv('DAEJEON_20190416_20190430.csv')#4월
#data2 = pd.read_csv('DAEJEON_20190516_20190531.csv')#5월
#data2 = pd.read_csv('DAEJEON_20190616_20190630.csv')#6월
#data2 = pd.read_csv('DAEJEON_20190716_20190731.csv')#7월
#data2 = pd.read_csv('DAEJEON_20190816_20190831.csv')#8월
#data2 = pd.read_csv('DAEJEON_20190916_20190930.csv')#9월
#data2 = pd.read_csv('DAEJEON_20191016_20191031.csv')#10월
#data2 = pd.read_csv('DAEJEON_20191116_20191130.csv')#11월
data2 = pd.read_csv('DAEJEON_20191216_20191231.csv')#12월
data2.head()

,id,x,y,timezn_cd,m00,m10,m15,m20,m25,m30,...,f35,f40,f45,f50,f55,f60,f65,f70,total,etl_ymd
0,56173942,332602,410243,0,0,0,0,0,2,2,...,0,1,3,1,1,0,0,0,17,20191217
1,56173942,332602,410243,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,8,20191226
2,56173942,332602,410243,0,0,0,1,1,0,1,...,0,1,1,1,1,0,0,0,16,20191231
3,56173942,332602,410243,0,0,0,1,1,0,1,...,0,1,1,1,0,0,0,0,19,20191228
4,56173942,332602,410243,0,0,0,1,1,1,0,...,1,0,1,0,0,0,0,0,18,20191222


In [27]:
#분석을 위해 하나의 파일로 합침
month_data = pd.concat([data, data2])

In [28]:
#필요없는 속성값만 지정해서 제거
del month_data['x']
del month_data['y']
del month_data['etl_ymd']
del month_data['timezn_cd']

In [29]:
#피봇 테이블 이용해 각 id별 유동인구 데이터 합침
pdf1 = pd.pivot_table(month_data,                
                     index = 'id',          
                     aggfunc = 'sum')

In [30]:
#조인해서 격자로 된 유동인구 데이터중 버퍼에 속한 격자 데이터만 가져옴
buffer_join = pd.merge(buffer,pdf1, left_on='cell_50_id', right_on='id', how='left')

In [31]:
#결측값은 유동인구 측정이 안되는 부분이니 0으로 처리 
buffer_join = buffer_join.fillna(0)

In [32]:
#구분 나눔
buffer_join['20대미만_남'] = buffer_join['m00'] +buffer_join['m10'] + buffer_join['m15']
buffer_join['20대_남'] = buffer_join['m20'] + buffer_join['m25']
buffer_join['30대_남'] = buffer_join['m30'] + buffer_join['m35']
buffer_join['40대_남'] = buffer_join['m40'] + buffer_join['m45']
buffer_join['50대_남'] = buffer_join['m50'] + buffer_join['m55']
buffer_join['60대이상_남'] = buffer_join['m60'] + buffer_join['m65'] +buffer_join['m70']

buffer_join['20대미만_여'] = buffer_join['f00'] +buffer_join['f10'] + buffer_join['f15']
buffer_join['20대_여'] = buffer_join['f20'] + buffer_join['f25']
buffer_join['30대_여'] = buffer_join['f30'] + buffer_join['f35']
buffer_join['40대_여'] = buffer_join['f40'] + buffer_join['f45']
buffer_join['50대_여'] = buffer_join['f50'] + buffer_join['f55']
buffer_join['60대이상_여'] = buffer_join['f60'] + buffer_join['f65'] +buffer_join['f70']

In [33]:
buffer_join['총합_남'] = buffer_join['20대미만_남']+buffer_join['20대_남']+buffer_join['30대_남']+buffer_join['40대_남']+buffer_join['50대_남']+buffer_join['60대이상_남']
buffer_join['총합_여'] = buffer_join['20대미만_여']+buffer_join['20대_여']+buffer_join['30대_여']+buffer_join['40대_여']+buffer_join['50대_여']+buffer_join['60대이상_여']


In [34]:
#필요없는 열들 삭제
del buffer_join['m00']
del buffer_join['m10']
del buffer_join['m15']
del buffer_join['m20']
del buffer_join['m25']
del buffer_join['m30']
del buffer_join['m35']
del buffer_join['m40']
del buffer_join['m45']
del buffer_join['m50']
del buffer_join['m55']
del buffer_join['m60']
del buffer_join['m65']
del buffer_join['m70']

del buffer_join['f00']
del buffer_join['f10']
del buffer_join['f15']
del buffer_join['f20']
del buffer_join['f25']
del buffer_join['f30']
del buffer_join['f35']
del buffer_join['f40']
del buffer_join['f45']
del buffer_join['f50']
del buffer_join['f55']
del buffer_join['f60']
del buffer_join['f65']
del buffer_join['f70']

In [35]:
"""
모든 데이터를 한번에 계산하려고 하면 중간에 램 부족으로 계산 과정이 날라가므로
데이터 백업을 위해 월별로 엑셀로 저장해둠
""" 

buffer_join.to_excel('버퍼안 유동인구/12월 버퍼안 유동인구.xlsx')

In [3]:
data1 = pd.read_excel('버퍼안 유동인구/1월 버퍼안 유동인구.xlsx')
data2 = pd.read_excel('버퍼안 유동인구/2월 버퍼안 유동인구.xlsx')
data3 = pd.read_excel('버퍼안 유동인구/3월 버퍼안 유동인구.xlsx')
data4 = pd.read_excel('버퍼안 유동인구/4월 버퍼안 유동인구.xlsx')
data5 = pd.read_excel('버퍼안 유동인구/5월 버퍼안 유동인구.xlsx')
data6 = pd.read_excel('버퍼안 유동인구/6월 버퍼안 유동인구.xlsx')
data7 = pd.read_excel('버퍼안 유동인구/7월 버퍼안 유동인구.xlsx')
data8 = pd.read_excel('버퍼안 유동인구/8월 버퍼안 유동인구.xlsx')
data9 = pd.read_excel('버퍼안 유동인구/9월 버퍼안 유동인구.xlsx')
data10 = pd.read_excel('버퍼안 유동인구/10월 버퍼안 유동인구.xlsx')
data11= pd.read_excel('버퍼안 유동인구/11월 버퍼안 유동인구.xlsx')
data12 = pd.read_excel('버퍼안 유동인구/12월 버퍼안 유동인구.xlsx')


In [4]:
year_data = pd.concat([data1, data2, data3, data4, data5, data6, data7, data8, data9, data10, data11, data12])

In [5]:
#피봇 테이블 이용해 각 id별 유동인구 데이터 합침
pdf1 = pd.pivot_table(year_data,                
                     index = 'cell_50_id',          
                     aggfunc = 'sum')

In [6]:
del pdf1['Unnamed: 0']
del pdf1['x_katech']
del pdf1['y_katech']

In [7]:
pdf1.head()

,20대_남,20대_여,20대미만_남,20대미만_여,30대_남,30대_여,40대_남,40대_여,50대_남,50대_여,60대이상_남,60대이상_여,total,총합_남,총합_여
cell_50_id,,,,,,,,,,,,,,,
58050960,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
58050961,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
58050962,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
58050963,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
58050964,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
# 일일 시간당 평균으로 나누지 않은 원본
buffer_join = pd.merge(buffer,pdf1, left_on='cell_50_id', right_on='cell_50_id', how='left')

In [9]:
buffer_join.to_excel('일일 평균으로 구하기 전.xlsx')

In [64]:
pdf1 = pdf1.iloc[: ,0:16] /(365*24)

In [66]:
buffer_join = pd.merge(buffer,pdf1, left_on='cell_50_id', right_on='cell_50_id', how='left')

In [67]:
buffer_join.to_excel('정제 결과.xlsx')